# 프롬프트 (Prompt)
- LLM한테 주는 입력 (지시, 맥락, 기억)
    - 지시: '~~ 해줘'
    - 맥락(context): 현재 지시를 위해 제공하는 추가 정보
    - 기억(memory): 지금까지 했던 대화 내용

In [ ]:
from dotenv import load_dotenv

load_dotenv()

True

## `promptTemplate`
- 단순히 1회성 명령 내릴 때 사용

In [9]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

llm = ChatOpenAI(model='gpt-4.1-nano')

In [ ]:
# 추후에 체인.invoke에서 {} 내부에 들어갈 말을 채워줘야 함
template = '{country}의 수도가 어디인가요?'

prompt = PromptTemplate.from_template(template)

# {}를 채우는 방법 (우리는 결국 chain.invoke로 쓰게 됨)
prompt.format(country='대한민국')

'대한민국의 수도가 어디인가요?'

In [11]:
chain = prompt | llm
chain.invoke({'country': '대한미국'}).content

'대한민국의 수도는 서울입니다.'

### Partial Variable (부분 변수)
- 프롬프트에서 매개변수 기본값 사용하기

In [16]:
template = '{c1}과 {c2}의 수도는 각각 어디인가요?'

prompt = PromptTemplate(
    template=template,
    input_variables=['c1'],
    partial_variables={
        'c2': '미국'
    }
)

prompt.format(c1='한국')
# prompt.format(c2='캐나다') <- 이건 에러 남

'한국과 미국의 수도는 각각 어디인가요?'

In [17]:
chain = prompt | llm
chain.invoke({'c1':'kor', 'c2':'us'}).content

'한국(코리아)의 수도는 서울입니다.  \n미국(United States)의 수도는 워싱턴 D.C.입니다.'

In [20]:
from datetime import datetime
print(datetime.now())

def get_today():
    return datetime.now().strftime('%b %d')

prompt = PromptTemplate(
    template='오늘 날짜는 {today}입니다. 오늘 생일인 유명인 {n}명을 생년월일과 함께 나열해 주세요',
    input_variables=['n'],
    partial_variables={
        'today': get_today
    }
)

prompt.format(n=3)

2025-09-01 15:16:31.270536


'오늘 날짜는 Sep 01입니다. 오늘 생일인 유명인 3명을 생년월일과 함께 나열해 주세요'

In [24]:
llm = ChatOpenAI(model='gpt-5-nano')

chain = prompt | llm
# 오늘 날짜 기준
print(chain.invoke({'n': 3}).content)
chain.invoke({'n': 3, 'today': 'Jan 2'}).content

오늘이 9월 1일인 만큼, 생일이 오늘인 유명인을 확인해 보겠습니다. 현재까지 확실히 알려진 인물은 다음과 같습니다.

- Zendaya — 1996년 9월 1일

나머지 두 명도 정확히 확인해 드리려면 인터넷 검색이 필요합니다. 계속 진행해도 될까요? 확인 후 세 명의 생년월일을 모두 정확히 적어 드리겠습니다.


'오늘이 1월 2일이 맞다면, 이 날 생일인 유명인 중 확실한 두 사람은 아래와 같습니다. (생년월일 포함)\n\n- 아이작 아시모프 (Isaac Asimov) — 1920년 1월 2일\n  미국의 과학소설 작가이자 비평가. 대표작으로는 Foundation 시리즈가 있습니다.\n\n- 케이트 보스워스 (Kate Bosworth) — 1983년 1월 2일\n  미국 배우로, 영화 "슈퍼맨 리턴즈" 등에 출연했습니다.\n\n세 번째 인물도 확실한 정보를 드리려면 웹 검색이 필요합니다. 원하시면 바로 찾아서 세 번째 인물까지 정확한 생년월일과 함께 다시 알려드리겠습니다. 특정 분야(영화/음악/스포츠 등) 선호가 있으면 더 맞춤으로 골라 드릴 수 있어요.'

## `ChatPromptTemplate`
- 채팅을 주고 받는 템플릿 생성용
- 대화 목록을 LLM에게 주입할 수 있음
- 하나의 Chat은 `role`과 `message`로 구성 된다

In [26]:
from langchain_core.prompts import ChatPromptTemplate

chat_prompt = ChatPromptTemplate.from_template('{country}의 수도는 어디?')
chat_prompt.format(country='한국')

'Human: 한국의 수도는 어디?'

In [34]:
chat_template = ChatPromptTemplate.from_messages(
    [
        # role - message
        ('system', '당신은 친절한 AI어시스트. 이름은 {name} 야.'),
        ('human', '반가워!'),
        ('ai', '무엇을 도와드릴까요?'),
        ('human', '{user_input}')
    ]
)

# format vs format_messages
messages_str = chat_template.format(name='gaida', user_input='이름이 뭐니?')
messages_cls = chat_template.format_messages(name='gaida', user_input='이름이 뭐니?')
print(messages_str)
print(messages_cls)

System: 당신은 친절한 AI어시스트. 이름은 gaida 야.
Human: 반가워!
AI: 무엇을 도와드릴까요?
Human: 이름이 뭐니?
[SystemMessage(content='당신은 친절한 AI어시스트. 이름은 gaida 야.', additional_kwargs={}, response_metadata={}), HumanMessage(content='반가워!', additional_kwargs={}, response_metadata={}), AIMessage(content='무엇을 도와드릴까요?', additional_kwargs={}, response_metadata={}), HumanMessage(content='이름이 뭐니?', additional_kwargs={}, response_metadata={})]


In [31]:
# 한덩어리 텍스트로 인식
llm.invoke(messages_str).content

# 실제 대화 내역으로 인지 (Langsmith가서 확인)
llm.invoke(messages_cls).content

'제 이름은 gaida예요. 반가워요! 무엇을 도와드릴까요?'

In [ ]:
chain = chat_template | llm | StrOutputParser()

chain.invoke({'name': '가이다', 'user_input': '내 이름은 민정. 너와 나의 이름점을 봐줘'})

'김민정님, 반가워요! 제 이름은 가이다이고, 님의 이름은 김민정이네요. 둘의 이름으로 재미있는 이름점(이름 궁합)을 한 번 보겠습니다. 이건 과학적 예언이 아니라 놀이로 보는 간단한 해석이에요.\n\n결과 요약\n- 총점: 78/100\n- 방법: 발음 흐름과 음운의 조화, 이름 길이의 균형 등을 간단히 비교해 산출한 점수예요.\n\n세부 해설\n- 발음 흐름: 8/10\n  가이다(가-이-다)와 김민정(김-민-정)을 잇는 흐름이 부드럽고, 두 이름 모두 모음이 비교적 편안하게 흘러듭니다. 서로 말할 때 리듬이 잘 맞는 편이에요.\n\n- 음운의 조화: 9/10\n  둘 다 공통으로 ㅣ 소리를 자주 쓰고, 첫 글자에서 ㄱ 소리가 연결됩니다. 또한 부드러운 모음들(ㅏ, ㅣ 등)이 조화를 이루죠. 다만 정의 ㅓ가 다소 차이를 만들어 주는 포인트가 있어 완벽한 일치보다는 따뜻한 차이가 있어요.\n\n- 균형성과 긍정성: 7/10\n  양쪽 모두 3음절 이름이라 길이의 균형이 잘 맞습니다. 다만 서로의 이름에 다양한 자음이 섞여 있어 활발한 상호작용이 기대됩니다.\n\n해석 요점\n- 두 이름은 서로의 분위기에 긍정적이고 친근한 에너지를 주고받는 조합으로 읽히는 편이에요.\n- 가이다의 창의적이고 다정한 느낌과 김민정님의 친근하고 정직한 느낌이 서로를 보완하는 경향이 있습니다.\n- 대화나 협업에서 서로의 차이를 존중하면 좋은 시너지를 낼 가능성이 큽니다.\n\n다른 방식으로도 더 자세히 알아보고 싶으신가요?\n- 더 정교한 방법으로: 자모 수, 초성/중성/종성의 수학적 매칭, 음양오행 배치 등으로 심화해 보기\n- 한자 의미 기반으로 읽기: 두 이름의 한자 구성과 뜻 비교\n- 또 다른 놀이: 이름의 음성 패턴(리듬, 악센트) 분석\n\n원하시는 방법이 있으면 알려 주세요. 원하시는 방식으로 좀 더 자세히 계산해 드리겠습니다.'

## 프롬프팅 방법
내가 원하는 답변을 구구절절 설명 X -> 예시를 주고 답변 형태를 유도
- Zero-shot prompting: 예시 없이 질문만 -> 바로 답변
- One-shot prompting: 예시 1개 + 질문 -> 모델이 예시를 모방해서 답변
- **Few-shot prompting**: 예시 여러개 + 질문 -> 예시들의 패턴을 일반화 해서 답변

In [40]:
from langchain_core.prompts.few_shot import FewShotPromptTemplate
from langchain_core.output_parsers import StrOutputParser

llm = ChatOpenAI(model='gpt-4.1-nano')

examples = [
    {
        "question": "스티브 잡스와 아인슈타인 중 누가 더 오래 살았나요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 스티브 잡스는 몇 살에 사망했나요?
중간 답변: 스티브 잡스는 56세에 사망했습니다.
추가 질문: 아인슈타인은 몇 살에 사망했나요?
중간 답변: 아인슈타인은 76세에 사망했습니다.
최종 답변은: 아인슈타인
""",
    },
    {
        "question": "네이버의 창립자는 언제 태어났나요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 네이버의 창립자는 누구인가요?
중간 답변: 네이버는 이해진에 의해 창립되었습니다.
추가 질문: 이해진은 언제 태어났나요?
중간 답변: 이해진은 1967년 6월 22일에 태어났습니다.
최종 답변은: 1967년 6월 22일
""",
    },
    {
        "question": "율곡 이이의 어머니가 태어난 해의 통치하던 왕은 누구인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 율곡 이이의 어머니는 누구인가요?
중간 답변: 율곡 이이의 어머니는 신사임당입니다.
추가 질문: 신사임당은 언제 태어났나요?
중간 답변: 신사임당은 1504년에 태어났습니다.
추가 질문: 1504년에 조선을 통치한 왕은 누구인가요?
중간 답변: 1504년에 조선을 통치한 왕은 연산군입니다.
최종 답변은: 연산군
""",
    },
    {
        "question": "올드보이와 기생충의 감독이 같은 나라 출신인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 올드보이의 감독은 누구인가요?
중간 답변: 올드보이의 감독은 박찬욱입니다.
추가 질문: 박찬욱은 어느 나라 출신인가요?
중간 답변: 박찬욱은 대한민국 출신입니다.
추가 질문: 기생충의 감독은 누구인가요?
중간 답변: 기생충의 감독은 봉준호입니다.
추가 질문: 봉준호는 어느 나라 출신인가요?
중간 답변: 봉준호는 대한민국 출신입니다.
최종 답변은: 예
""",
    },
]

example_prompt = PromptTemplate.from_template(
    "Question:\n{question}\nAnswer:\n{answer}"
)

prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Question:\n{question}\nAnswer:",
    input_variables=["question"],
)

question = "Google이 창립된 연도에 Bill Gates의 나이는 몇 살인가요?"

chain = prompt | llm | StrOutputParser()

res = chain.invoke({'question': question})

print(res)

이 질문에 추가 질문이 필요한가요: 예.  
추가 질문: Google이 언제 창립되었나요?  
중간 답변: Google은 1998년에 창립되었습니다.  
추가 질문: Bill Gates는 언제 태어났나요?  
중간 답변: Bill Gates는 1955년 10월 28일에 태어났습니다.  
최종 답변은: 42세입니다.


## `langchain-hub`
다양한 사용자들이 업로드한 프롬프트를 받아서 활용

In [39]:
from langchain import hub

prompt = hub.pull('hwchase17/react')
print(prompt.template)

Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}
